# Imports

In [ ]:
import os
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from PIL import Image, ImageDraw, ImageFont

import pickle

from scipy.spatial.transform import Rotation as R
from sklearn.metrics.pairwise import cosine_similarity

import supervision as sv
import open3d as o3d
from cuml.cluster import DBSCAN
import cupy as cp

In [ ]:
# TODO: Install pytorch and add imports

# Setup

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
# TODO: Add correct paths

imgs_dir = "/scratch/laksh.nanwani/instance_slam/JmbYfDe2QKZ/rgb"
depth_dir = "/scratch/laksh.nanwani/instance_slam/JmbYfDe2QKZ/depth/"
pose_dir = "/scratch/laksh.nanwani/instance_slam/JmbYfDe2QKZ/pose/"
img_dict_dir = "/home2/kumaradi.gupta/instance-map/img_dict.pkl"

In [ ]:
req_tags = ['chair', 'clock', 'closet', 'stool', 'table', 'couch', 'fan', 'door']

colors = [
    (255, 105, 97),  # Pastel Red
    (255, 179, 71),  # Pastel Orange
    (253, 253, 150),  # Pastel Yellow
    (119, 221, 119),  # Pastel Green
    (203, 153, 201),  # Pastel Purple
    (174, 198, 207),  # Pastel Blue
    (255, 209, 220),  # Pastel Pink
    (0, 255, 0)       # Green
]

def normalize_color(color_255):
    return tuple(val/255.0 for val in color_255)

In [ ]:
# Load from pickle file
with open(img_dict_dir, 'rb') as file:
    img_dict = pickle.load(file)


# 3D AABB Calc

In [ ]:
'''
img_dict = {img_name: {img_path: str,
                        ram_tags: list_of_str,
                        objs: {0: {bbox: [x1, y1, x2, y2],
                                    phrase: str,
                                    clip_embed: [1, 1024]},
                                    dino_embed: [1, 1024]},
                                    mask: [h, w],
                                    prob: float,
                                    aabb: arr}
                                1: {...},
                        }
            img_name: {...},
            }
'''

# go through the img_name and their data
# open the depth associated with an image from the depth folder (.npy file)

def get_depth(img_name):
    depth_path = os.path.join(depth_dir, img_name + '.npy')
    depth = np.load(depth_path)
    return depth

def get_pose(img_name):
    pose_path = os.path.join(pose_dir, img_name + '.txt')
    with open(pose_path, 'r') as f:
        pose = f.read().split()
        pose = np.array(pose).astype(np.float32)
    return pose

def get_sim_cam_mat_with_fov(h, w, fov):
    cam_mat = np.eye(3)
    cam_mat[0, 0] = cam_mat[1, 1] = w / (2.0 * np.tan(np.deg2rad(fov / 2)))
    cam_mat[0, 2] = w / 2.0
    cam_mat[1, 2] = h / 2.0
    return cam_mat

In [ ]:
def create_point_cloud(depth, mask, cam_mat, pose, color=(1, 0, 0), camera_height=0.9):
    """
    Generates a point cloud from a depth image, camera intrinsics, mask, and pose.
    Only points within the mask and with valid depth are added to the cloud.
    Points are colored using the specified color.
    """

    # Reproject the depth to 3D space
    rows, cols = np.where(mask)
    points2d = np.vstack([cols, rows, np.ones_like(rows)])
    depth_values = depth[rows, cols]
    cam_mat_inv = np.linalg.inv(cam_mat)
    points3d_cam = cam_mat_inv @ points2d * depth_values

    # Parse the pose
    pos = np.array(pose[:3], dtype=float).reshape((3, 1))
    quat = pose[3:]
    rot = R.from_quat(quat).as_matrix()

    # Apply rotation correction, to match the orientation z: backward, y: upward, and x: right
    rot_ro_cam = np.eye(3)
    rot_ro_cam[1, 1] = -1
    rot_ro_cam[2, 2] = -1
    rot = rot @ rot_ro_cam

    # Apply position correction
    pos[1] += camera_height

    # Create the pose matrix
    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = rot
    pose_matrix[:3, 3] = pos.reshape(-1)

    # Transform the points to global frame
    points3d_homo = np.vstack([points3d_cam, np.ones((1, points3d_cam.shape[1]))])
    points3d_global_homo = pose_matrix @ points3d_homo
    points3d_global = points3d_global_homo[:3, :]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d_global.T)

    # Assign color to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points3d_global.shape[1], 1)))

    return pcd


In [ ]:
def get_obj_bounds(obj_pc):
    aabb = obj_pc.get_axis_aligned_bounding_box()
    min_bound = aabb.min_bound
    max_bound = aabb.max_bound
    obj_bounds = np.vstack((min_bound, max_bound))
    return obj_bounds

def iou_3d(box1, box2):
    min_intersection = np.maximum(box1[0], box2[0])
    max_intersection = np.minimum(box1[1], box2[1])

    # Compute the intersection volume
    intersection_dims = np.maximum(max_intersection - min_intersection, 0)
    intersection_volume = np.prod(intersection_dims)

    # Compute the volumes of the individual bounding boxes
    volume_box1 = np.prod(box1[1] - box1[0])
    volume_box2 = np.prod(box2[1] - box2[0])

    # Compute the union volume
    union_volume = volume_box1 + volume_box2 - intersection_volume

    # Compute the IoU
    iou = intersection_volume / union_volume if union_volume != 0 else 0
    return iou

In [ ]:
def filter_noise_dbscan(point_cloud, eps=0.1, min_points=15):
    # Apply DBSCAN clustering to the point cloud
    # with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    #     labels = np.array(point_cloud.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))

    labels = np.array(point_cloud.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))

    # Find the largest cluster
    max_label = max(labels, key=list(labels).count)

    # Filter the point cloud to include only the largest cluster
    filtered_points = np.asarray(point_cloud.points)[labels == max_label]
    filtered_colors = np.asarray(point_cloud.colors)[labels == max_label]

    # Create a new point cloud with only the largest cluster
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

    return filtered_pcd


In [ ]:
def fast_DBSCAN(point_cloud_o3d, eps=0.01, min_samples=20):
    # Convert Open3D point cloud to NumPy array
    points_np = np.asarray(point_cloud_o3d.points)

    # Convert NumPy array to CuPy array for GPU computations
    points_gpu = cp.asarray(points_np)

    # Create a DBSCAN instance with cuML
    dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)

    # Fit the model to the GPU data
    dbscan_model.fit(points_gpu)

    # Get the labels for the clusters
    labels_gpu = dbscan_model.labels_

    # Convert the labels back to a NumPy array
    labels = cp.asnumpy(labels_gpu)

    # Filter the points that are part of a cluster (label != -1)
    clustered_points = points_np[labels != -1]

    # Create a new Open3D point cloud with the clustered points
    filtered_point_cloud_o3d = o3d.geometry.PointCloud()
    filtered_point_cloud_o3d.points = o3d.utility.Vector3dVector(clustered_points)

    return filtered_point_cloud_o3d


In [ ]:
for img_id, img_data in tqdm(img_dict.items()):

  if len(img_data['objs']) == 0:
    continue

  # img_path = img_data['img_path']
  # img = cv2.imread(img_path)

  depth = get_depth(img_id)
  pose = get_pose(img_id)
  cam_mat = get_sim_cam_mat_with_fov(depth.shape[0], depth.shape[1], fov=90)

  for obj_id, obj_data in img_data['objs'].items():
    mask = obj_data['mask']
    phrase = obj_data['phrase']
    phrase = phrase.split()[0]

    pcd = create_point_cloud(depth, mask, cam_mat, pose)

    obj_data['aabb'] = get_obj_bounds(pcd)

In [ ]:
# Save updated img_dict with aabb

with open('/content/drive/MyDrive/habitat_output/img_dict.pkl', 'wb') as file:
    pickle.dump(img_dict, file)

# Cluster Objects

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from collections import defaultdict

# Function to merge two clusters
def merge_data(cluster1, cluster2):
    # Compute the union of the bounding boxes
    obj_bounds1 = cluster1['aabb']
    obj_bounds2 = cluster2['aabb']

    min_bound = np.minimum(obj_bounds1[0], obj_bounds2[0])
    max_bound = np.maximum(obj_bounds1[1], obj_bounds2[1])

    # Merge source IDs: source_ids: [(img_id, obj_id), ...]
    source_ids = cluster1['source_ids'] + cluster2['source_ids']
    count = len(source_ids)

    # Average the embeddings
    avg_dino_embed = (np.array(cluster1['dino_embed']) * len(cluster1['source_ids']) +
                      np.array(cluster2['dino_embed']) * len(cluster2['source_ids'])) / count
    avg_clip_embed = (np.array(cluster1['clip_embed']) * len(cluster1['source_ids']) +
                      np.array(cluster2['clip_embed']) * len(cluster2['source_ids'])) / count
    return {
        'aabb': np.vstack((min_bound, max_bound)),
        'source_ids': source_ids,
        'dino_embed': avg_dino_embed,
        'clip_embed': avg_clip_embed
    }

In [ ]:
#Old clustering algo: works but is wrong

def cluster_objects(img_dict, iou_threshold=0.5):
    # A dictionary to store object-level data
    obj_dict = {}
    cluster_counter = 0

    # Cluster heirarchy 1
    # Iterate through the images and objects
    for img_id, img_data in tqdm(img_dict.items()):

        if len(img_data['objs']) == 0:
          continue

        for obj_id, obj_data in img_data['objs'].items():
          
            # Skip the object if aabb does not exist, in case of partial data only
            if 'aabb' not in obj_data:
                continue

            phrase = obj_data['phrase']
            phrase = phrase.split()[0]

            if phrase == 'armchair': #for testing only
              phrase = 'chair'

            # Initialize the phrase entry if not present
            if phrase not in obj_dict:
                obj_dict[phrase] = {}

            # Iterate through existing clusters for this phrase
            found_cluster = False
            for cluster_id, cluster_data in list(obj_dict[phrase].items()):
                # Get the IoU for the current box and the cluster
                iou = iou_3d(obj_data['aabb'], cluster_data['aabb'])

                if iou > iou_threshold:
                    # If IoU is above the threshold, update the cluster
                    found_cluster = True
                    obj_dict[phrase][cluster_id] = merge_data(cluster_data, {
                        'aabb': obj_data['aabb'],
                        'source_ids': [(img_id, obj_id)],
                        'dino_embed': obj_data['dino_embed'],
                        'clip_embed': obj_data['clip_embed']
                    })

                    # Check for possible merges with other clusters
                    for other_cluster_id, other_cluster_data in list(obj_dict[phrase].items()):
                        if other_cluster_id != cluster_id:
                            iou_merge = iou_3d(obj_dict[phrase][cluster_id]['aabb'],
                                               other_cluster_data['aabb'])

                            if iou_merge > iou_threshold:
                                merged_cluster = merge_data(obj_dict[phrase][cluster_id],
                                                            other_cluster_data)
                                obj_dict[phrase][cluster_id] = merged_cluster
                                del obj_dict[phrase][other_cluster_id]

                    break

            if not found_cluster:
                # If no cluster is found, add a new cluster
                cluster_id = len(obj_dict[phrase]) # Doesn't work because of arbitrary merging
                obj_dict[phrase][cluster_id] = {
                    'aabb': obj_data['aabb'],
                    'source_ids': [(img_id, obj_id)],
                    'dino_embed': obj_data['dino_embed'],
                    'clip_embed': obj_data['clip_embed']
                }
                cluster_counter += 1

    # Not good: Final merge check among remaining clusters
    # for phrase, clusters in obj_dict.items():
    #     cluster_ids = list(clusters.keys())
    #     i = 0
    #     while i < len(cluster_ids):
    #         cluster_id = cluster_ids[i]
    #         cluster_data = clusters[cluster_id]
    #         merged = False
    #         for other_cluster_id, other_cluster_data in list(clusters.items()):
    #             if other_cluster_id != cluster_id:
    #                 iou_merge = iou_3d(cluster_data['aabb'], other_cluster_data['aabb'])
    #                 if iou_merge > iou_threshold:
    #                     merged_cluster = merge_data(cluster_data, other_cluster_data)
    #                     clusters[cluster_id] = merged_cluster
    #                     del clusters[other_cluster_id]
    #                     cluster_ids.remove(other_cluster_id)
    #                     merged = True
    #                     break
    #         if not merged:
    #             i += 1

    return obj_dict


In [ ]:
#New clustering algo: correct but the results are not great

def custom_distance(obj1, obj2):

    iou_weight = 0.7
    dino_weight = 0.3

    iou = iou_3d(obj1['aabb'], obj2['aabb'])

    dino_similarity = cosine_similarity(obj1['dino_embed'].reshape(1, -1),
                                        obj2['dino_embed'].reshape(1, -1))[0][0]

    distance = ((1 - iou) * iou_weight) + ((1 - dino_similarity) * dino_weight)

    if distance < 0: # Happening in cases where iou and dino sim = 1
      print(f"Obj1: {obj1['source_ids']}, Obj2: {obj2['source_ids']}")
      distance = 0

    return distance

def cluster_objects(img_dict, dist_threshold=0.7):
    # Group objects by phrase
    phrase_objects = defaultdict(list)
    for img_id, img_data in img_dict.items():
        for obj_id, obj_data in img_data['objs'].items():

            obj_data_with_source = obj_data.copy()
            obj_data_with_source['source_ids'] = [(img_id, obj_id)]  # Add source information

            phrase = obj_data_with_source['phrase'].split()[0]
            phrase_objects[phrase].append(obj_data_with_source)

    # Dictionary to store the final merged clusters
    merged_clusters = {}

    # Perform clustering for each phrase
    for phrase, objects_list in tqdm(phrase_objects.items()):

        # Compute pairwise distances and convert to condensed form
        n = len(objects_list)
        distances = np.zeros((n, n))
        for i in range(n):
            for j in range(i + 1, n):  # Only calculate upper triangular part
                distances[i, j] = custom_distance(objects_list[i], objects_list[j])
                distances[j, i] = distances[i, j]  # Symmetric matrix
        distances = squareform(distances)

        # Perform hierarchical clustering
        Z = linkage(distances, method='single')

        # Cut the dendrogram to form final clusters
        cluster_labels = fcluster(Z, t=dist_threshold, criterion='distance')

        # Merge clusters for this phrase
        phrase_clusters = {}
        for idx, label in enumerate(cluster_labels):
            if label not in phrase_clusters:
                phrase_clusters[label] = objects_list[idx]
            else:
                # Merge data using your existing merge_data function
                phrase_clusters[label] = merge_data(phrase_clusters[label], objects_list[idx])

        # Store the merged clusters for this phrase
        merged_clusters[phrase] = phrase_clusters

    return merged_clusters


In [ ]:
obj_dict = cluster_objects(img_dict, dist_threshold=0.60)

In [ ]:
print(len(obj_dict['chair']), len(obj_dict['table']))

In [ ]:
def create_obj_point_clouds(obj_dict, img_dict, voxel_size=0.05, color=(1, 0, 0)):
    # Ensure the outputs directory exists
    if not os.path.exists('outputs'):
        os.mkdir('outputs')

    # Iterate through the phrases and clusters
    for phrase, clusters in tqdm(obj_dict.items()):
        # Create a directory for this phrase if it doesn't exist
        phrase_dir = os.path.join('outputs', phrase)
        if not os.path.exists(phrase_dir):
            os.mkdir(phrase_dir)

        color = colors[req_tags.index(phrase)]
        color = normalize_color(color)

        # Iterate through the clusters
        for cluster_id, cluster_data in tqdm(clusters.items()):
            # Initialize an empty point cloud to accumulate points from all source IDs
            final_pcd = o3d.geometry.PointCloud()

            # Iterate through the source IDs for this cluster
            for img_id, obj_id in cluster_data['source_ids']:
                # Get the required information
                depth = get_depth(img_id)
                pose = get_pose(img_id)
                cam_mat = get_sim_cam_mat_with_fov(depth.shape[0], depth.shape[1], fov=90)
                mask = img_dict[img_id]['objs'][obj_id]['mask']

                # Create the point cloud for this source ID
                pcd = create_point_cloud(depth, mask, cam_mat, pose, color=color)
                # Combine with the final point cloud
                final_pcd += pcd

            # Downsample and filter the final point cloud
            final_pcd = final_pcd.voxel_down_sample(voxel_size=voxel_size)
            final_pcd = filter_noise_dbscan(final_pcd, eps=0.1, min_points=20)


            # Save the final point cloud for this cluster
            file_path = os.path.join(phrase_dir, f'{cluster_id}.ply')
            o3d.io.write_point_cloud(file_path, final_pcd)


In [ ]:
%cd '/content/drive/MyDrive/habitat_output'

create_obj_point_clouds(obj_dict, img_dict, voxel_size=0.005)

# Experiments

In [ ]:
# Compare the embeddings of all images, first object instance

embedding_type = 'dino_embed'
# Sort the img_ids
sorted_img_ids = sorted(img_dict.keys(), key=lambda x: [int(i) for i in x.split('_')])

# Extract the embeddings in the sorted order
embeddings = [img_dict[img_id]['objs'][0][embedding_type] for img_id in sorted_img_ids]

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# Visualize the similarity matrix as a heatmap
plt.figure(figsize=(8, 8))
plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
plt.colorbar(label='Cosine Similarity')
plt.title('Clip Embedding Similarity')

# Add the similarity scores as text labels
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix)):
        plt.text(j, i, f'{similarity_matrix[i, j]:.2f}', ha='center', va='center', color='white' if similarity_matrix[i, j] < 0.5 else 'black')

plt.xticks(range(len(img_dict)), sorted_img_ids, rotation=90)
plt.yticks(range(len(img_dict)), sorted_img_ids)
plt.show()



In [ ]:
#Code to visualize separate masks for a single image

img_id = 'JmbYfDe2QKZ_1'

# Load the image
img_path = img_dict[img_id]['img_path']
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Get the object data for the image
objs = img_dict[img_id]['objs']

# Loop through each object in the image
for obj_index, obj_data in objs.items():
    # Get the mask for the object
    mask = obj_data['mask']

    # If the mask isn't a numpy array, convert it to one
    if not isinstance(mask, np.ndarray):
        mask = mask.numpy()

    # Apply the mask to the image. This assumes your mask is binary (0s and 1s).
    # If it isn't, you may need to adjust this step accordingly.
    masked_image = img * np.expand_dims(mask, axis=2)

    # Display the masked image
    plt.figure()
    plt.imshow(masked_image)
    plt.title(f"Object {obj_index}")
    plt.show()


In [ ]:
#Code to draw the segmentation mask on the image
from torchvision.utils import draw_segmentation_masks

img_id = 'JmbYfDe2QKZ_123'

# Load the image
img_path = img_dict[img_id]['img_path']
img = Image.open(img_path)  # Load with PIL to avoid needing to convert color spaces

transform = transforms.Compose([transforms.ToTensor()])
img_tensor = transform(img)
img_tensor = (img_tensor * 255).byte()


masks = []
mask_colors = []

objs = img_dict[img_id]['objs']

for obj_index, obj_data in objs.items():
    mask = obj_data['mask']

    phrase = obj_data['phrase']
    phrase = phrase.split()[0] #Use only the first phrase

    # If the mask isn't a numpy array, convert it to one
    if not isinstance(mask, np.ndarray):
        mask = mask.numpy()

    # Add the mask to the list of masks
    masks.append(mask)
    #Cheeck the color for the mask and add it to the list
    color = colors[req_tags.index(phrase)]
    mask_colors.append(color)

# Convert the masks to a boolean tensor
masks_tensor = torch.tensor(masks, dtype=torch.bool)

# Draw all masks on the image
result = draw_segmentation_masks(img_tensor, masks_tensor, colors=mask_colors)

# Convert the result tensor back to a PIL image, transpose the dimensions back to H,W,C for PIL
result_img = Image.fromarray(result.permute(1, 2, 0).byte().cpu().numpy())

# Display the image with all masks
plt.imshow(result_img)
plt.show()


In [ ]:
# Code to check if there are multiple phrases for a single object

for img_id, img_data in tqdm(img_dict.items()):
    objs_data = img_data['objs']
    for obj_id, obj_data in objs_data.items():
        phrase = obj_data['phrase']
        words = phrase.split()  # Splits the phrase into words using whitespace as the separator
        if len(words) > 1:  # Checks if there are more than one word
            print(f'{img_id}_{obj_id}: {phrase}')


In [ ]:
# Code to print the similarity of dino embeds for all objects of an image

img_id = 'JmbYfDe2QKZ_123'
objs_data = img_dict[img_id]['objs']

dino_embeds = [obj_data['dino_embed'] for obj_id, obj_data in objs_data.items()]
phrases = [obj_data['phrase'] for obj_id, obj_data in objs_data.items()]

# Reshaping the embeds to fit the input shape for cosine_similarity
dino_embeds_matrix = np.array(dino_embeds).reshape(-1, 768)

# Calculating the cosine similarity
similarity_matrix = cosine_similarity(dino_embeds_matrix)

# print("Cosine Similarities:")
# print(similarities)

# Visualize the similarity matrix as a heatmap
plt.figure(figsize=(8, 8))
plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
plt.colorbar(label='Cosine Similarity')
plt.title('Dino Embedding Similarity')

# Add the similarity scores as text labels
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix)):
        plt.text(j, i, f'{similarity_matrix[i, j]:.2f}', ha='center', va='center', color='white' if similarity_matrix[i, j] < 0.5 else 'black')

plt.xticks(range(len(phrases)), phrases, rotation=90)
plt.yticks(range(len(phrases)), phrases)
plt.show()


In [ ]:
# Visualize 3D bounding boxes without DBSCAN
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

img_ids = ['JmbYfDe2QKZ_123']

# Normalize the colors to (0-1) range
colors_normalized = [(r/255, g/255, b/255) for r, g, b in colors]

# Create a mapping of phrases to colors
color_mapping = {phrase: color for phrase, color in zip(req_tags, colors_normalized)}
default_color = 'gray' # Default color if the phrase is not in the mapping

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Finding the global min and max bounds across all bounding boxes
global_min_bound = np.array([float('inf')] * 3)
global_max_bound = np.array([float('-inf')] * 3)

for img_id in img_ids:
  obj_data = img_dict[img_id]['objs']

  for obj_id, data in tqdm(obj_data.items()):

      bounds = data['aabb']

      phrase = data['phrase']
      phrase = phrase.split()[0]

      # if phrase != 'chair':
      #     continue

      color = color_mapping.get(phrase, default_color)

      min_bound, max_bound = bounds
      min_bound = [min_bound[0], min_bound[2], min_bound[1]]
      max_bound = [max_bound[0], max_bound[2], max_bound[1]]

      global_min_bound = np.minimum(global_min_bound, min_bound)
      global_max_bound = np.maximum(global_max_bound, max_bound)

      # Define vertices of the bounding box
      vertices = [
          [min_bound[0], min_bound[1], min_bound[2]],
          [max_bound[0], min_bound[1], min_bound[2]],
          [max_bound[0], max_bound[1], min_bound[2]],
          [min_bound[0], max_bound[1], min_bound[2]],
          [min_bound[0], min_bound[1], max_bound[2]],
          [max_bound[0], min_bound[1], max_bound[2]],
          [max_bound[0], max_bound[1], max_bound[2]],
          [min_bound[0], max_bound[1], max_bound[2]]
      ]

      # Define faces using vertices
      faces = [
          [vertices[0], vertices[1], vertices[2], vertices[3]],
          [vertices[4], vertices[5], vertices[6], vertices[7]],
          [vertices[0], vertices[1], vertices[5], vertices[4]],
          [vertices[2], vertices[3], vertices[7], vertices[6]],
          [vertices[1], vertices[2], vertices[6], vertices[5]],
          [vertices[4], vertices[7], vertices[3], vertices[0]]
      ]

      # Plot the faces
      for face in faces:
          poly3d = [[vertice[0], vertice[1], vertice[2]] for vertice in face]
          ax.add_collection3d(Poly3DCollection([poly3d], facecolors=color, linewidths=1, edgecolors='k', alpha=0.25))


# Setting the axes limits based on the global bounds
ax.set_xlim([global_min_bound[0], global_max_bound[0]])
ax.set_ylim([global_min_bound[1], global_max_bound[1]])
ax.set_zlim([global_min_bound[2], global_max_bound[2]])
plt.show()


In [ ]:
# Visualize 3D bounding boxes with DBSCAN
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

img_ids = ['JmbYfDe2QKZ_123']

depth = get_depth(img_id)
pose = get_pose(img_id)
cam_mat = get_sim_cam_mat_with_fov(depth.shape[0], depth.shape[1], fov=90)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Normalize the colors to (0-1) range
colors_normalized = [(r/255, g/255, b/255) for r, g, b in colors]

# Create a mapping of phrases to colors
color_mapping = {phrase: color for phrase, color in zip(req_tags, colors_normalized)}
default_color = 'gray' # Default color if the phrase is not in the mapping

# Finding the global min and max bounds across all bounding boxes
global_min_bound = np.array([float('inf')] * 3)
global_max_bound = np.array([float('-inf')] * 3)

for img_id in img_ids:
  obj_data = img_dict[img_id]['objs']
  for obj_id, data in tqdm(obj_data.items()):

      #Reconstruct pointcloud
      mask = data['mask']
      phrase = data['phrase']
      phrase = phrase.split()[0]

      color = color_mapping.get(phrase, default_color)

      pcd = create_point_cloud(depth, mask, cam_mat, pose)

      filtered_point_cloud = fast_DBSCAN(pcd)
      
      pcd = pcd.voxel_down_sample(voxel_size=0.005)
      pcd = filter_noise_dbscan(pcd, eps=0.1, min_points=15)

      bounds = get_obj_bounds(pcd)
      # bounds = data['aabb']
      min_bound, max_bound = bounds

      min_bound = [min_bound[0], min_bound[2], min_bound[1]]
      max_bound = [max_bound[0], max_bound[2], max_bound[1]]

      global_min_bound = np.minimum(global_min_bound, min_bound)
      global_max_bound = np.maximum(global_max_bound, max_bound)

      # Define vertices of the bounding box
      vertices = [
          [min_bound[0], min_bound[1], min_bound[2]],
          [max_bound[0], min_bound[1], min_bound[2]],
          [max_bound[0], max_bound[1], min_bound[2]],
          [min_bound[0], max_bound[1], min_bound[2]],
          [min_bound[0], min_bound[1], max_bound[2]],
          [max_bound[0], min_bound[1], max_bound[2]],
          [max_bound[0], max_bound[1], max_bound[2]],
          [min_bound[0], max_bound[1], max_bound[2]]
      ]

      # Define faces using vertices
      faces = [
          [vertices[0], vertices[1], vertices[2], vertices[3]],
          [vertices[4], vertices[5], vertices[6], vertices[7]],
          [vertices[0], vertices[1], vertices[5], vertices[4]],
          [vertices[2], vertices[3], vertices[7], vertices[6]],
          [vertices[1], vertices[2], vertices[6], vertices[5]],
          [vertices[4], vertices[7], vertices[3], vertices[0]]
      ]

      # Plot the faces
      for face in faces:
          poly3d = [[vertice[0], vertice[1], vertice[2]] for vertice in face]
          ax.add_collection3d(Poly3DCollection([poly3d], facecolors=color, linewidths=1, edgecolors='k', alpha=0.25))


# Setting the axes limits based on the global bounds
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_xlim([global_min_bound[0], global_max_bound[0]])
ax.set_ylim([global_min_bound[1], global_max_bound[1]])
ax.set_zlim([global_min_bound[2], global_max_bound[2]])
plt.show()


In [ ]:
img_id = 'JmbYfDe2QKZ_123'
depth_data = get_depth(img_id)

# Visualize the depth data as a heatmap
plt.figure(figsize=(8, 8))
plt.imshow(depth_data, cmap='viridis')  # You can choose different colormaps like 'gray', 'plasma', etc.
plt.colorbar(label='Depth Value')
plt.title('Depth Visualization')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()
